In [17]:
import os
import pandas as pd

# Parameters
days_before_event = 3
days_after_event = 3

RTD_higher_than = -70
voltage_lower_than = 200

remove_from_date = '2017-11-01'
remove_until_date = '2018-08-01'

# List of error events that indicate a problem in the freezer
error_events = [
    "setpoint_change",                     # Wrong setpoint
    "Bad insulation",                      # Bad insulation
    "door_adjustment",                     # Door tightness
    "High condensation water",             # High condensation
    "display",                             # Data missing
    "instability",                         # Unstable operation
    "compressor_stage_1 malfunctional",    # 1st stage compressor
    "compressor_stage_2 malfunctional",    # 2nd stage compressor
    "electric_wiring",                     # Electrical malfunction
    "Refrigerant leakage at stage 1",      # Refrigerant leakage
    "Refrigerant leakage at stage 2"
]

# Path to the directory containing freezer data
path = '/Users/muhammadhussain/Desktop/Data/Revco/'  # Replace with the actual path to your dataset

# Loop through all files in the directory
for file_name in os.listdir(path):
    # Check if the file matches the expected format
    if file_name.endswith('_temp.parquet'):
        # Extract the freezer number from the file name
        Freezer_number = file_name.split('_')[0]
        
        # Load the Parquet file
        df = pd.read_parquet(os.path.join(path, file_name))
        
        # Check if 'main_fault' column exists
        if 'main_fault' not in df.columns:
            print(f"'main_fault' column not found in {file_name}, skipping...")
            continue  # Skip this file and move to the next
        
        # Ensure 'Datetime' is in datetime format
        df['Datetime'] = pd.to_datetime(df['Datetime'])
        
        # **Step 1**: Filter out rows based on `RTD` and `Mains Voltage`
        df = df[(df['RTD'] <= RTD_higher_than) & (df['Mains Voltage'] >= voltage_lower_than)]
        
        # **Step 2**: Remove rows between dates (if needed)
        # df = df[~((df['Datetime'] >= remove_from_date) & (df['Datetime'] <= remove_until_date))]
        
        # **Step 3**: Identify the indices where a main fault has occurred
        fault_indices = df[df['main_fault'].notnull()].index
        
        # Sets to collect indices for rows to keep in the time window
        indices_to_collect_error = set()
        indices_to_collect_non_error = set()
        
        # Loop through each main fault
        for idx in fault_indices:
            # Get the main fault timestamp and fault value
            fault_time = df.loc[idx, 'Datetime']
            fault_value = df.loc[idx, 'main_fault']  # Get the fault value at the current index
            
            # Define the time window around the main fault
            start_time = fault_time - pd.Timedelta(days=days_before_event)
            end_time = fault_time + pd.Timedelta(days=days_after_event)
            
            # Select rows within the time window
            mask = (df['Datetime'] >= start_time) & (df['Datetime'] <= end_time)
            window_indices = df[mask].index
            
            # Check if the main fault is an error event
            if fault_value in error_events:
                # Assign the same main_fault value to all rows in the window
                df.loc[window_indices, 'main_fault'] = fault_value
                indices_to_collect_error.update(window_indices)
            else:
                indices_to_collect_non_error.update(window_indices)
        
        # Create DataFrames for error events and clean data
        df_around_events = df.loc[list(indices_to_collect_error)].copy()  # Data with error events in the time window
        df_cleaned = df.drop(indices_to_collect_error.union(indices_to_collect_non_error))  # Remove all handled rows from main DataFrame
        
        # **Step 6**: Save both DataFrames
        df_around_events.to_parquet(
            os.path.join('/Users/muhammadhussain/Desktop/Data/filter/', f'around_events_data_{Freezer_number}.parquet'), index=False
        )
        df_cleaned.to_parquet(
            os.path.join('/Users/muhammadhussain/Desktop/Data/filter/', f'cleaned_data_{Freezer_number}.parquet'), index=False
        )


'main_fault' column not found in 806275_temp.parquet, skipping...
'main_fault' column not found in 806267_temp.parquet, skipping...
'main_fault' column not found in 806027_temp.parquet, skipping...
'main_fault' column not found in 806280_temp.parquet, skipping...
'main_fault' column not found in 806016_temp.parquet, skipping...
'main_fault' column not found in 806034_temp.parquet, skipping...
'main_fault' column not found in 806021_temp.parquet, skipping...
'main_fault' column not found in 806830_temp.parquet, skipping...
'main_fault' column not found in 806268_temp.parquet, skipping...
'main_fault' column not found in 806019_temp.parquet, skipping...
'main_fault' column not found in 806270_temp.parquet, skipping...
'main_fault' column not found in 806028_temp.parquet, skipping...
'main_fault' column not found in 806271_temp.parquet, skipping...
'main_fault' column not found in 806829_temp.parquet, skipping...


In [18]:
import pandas as pd
import glob
import time
from sklearn.preprocessing import MinMaxScaler
import os

path = '/Users/muhammadhussain/Desktop/Data/filter/'

def process_parquet_files(path, output_file, window_size=60):
    start_time = time.time()  # Track total time

    # Modelnumre, der skal ekskluderes
    excluded_models = {"806026", "806030", "806031", "806276", "806279"}

    # Step 1: Load all parquet files matching the pattern
    around_event_files = glob.glob(path + "around_events_data_*.parquet")
    cleaned_data_file = os.path.join(path, "cleaned_data_806278.parquet")

    if not around_event_files:
        print(f"No 'around_events_data_*' files found in the directory {path}. Exiting.")
        return

    # Step 2: Initialize a list to store processed windows
    time_windows = []
    total_groups = 0  # Counter for total number of groups across all models
    skipped_groups_total = 0  # Counter for total skipped groups across all models

    # Step 3: Process "around_events_data_*.parquet" files
    for file in around_event_files:
        model_number = os.path.basename(file).split('_')[3].split('.')[0]

        # Check if the model number is in the excluded list
        if model_number in excluded_models:
            print(f"Skipping model {model_number} (excluded)")
            continue

        df = pd.read_parquet(file)
        df['Datetime'] = pd.to_datetime(df['Datetime'])

        # Ensure the data is sorted by time
        df = df.sort_values('Datetime').reset_index(drop=True)

        # Step 4: Identify continuous sequences
        df['time_diff'] = df['Datetime'].diff().dt.total_seconds()
        df['is_continuous'] = (df['time_diff'].between(59, 61)) | (df.index == 0)
        df['sequence_group'] = (~df['is_continuous']).cumsum()

        # Step 5: Count total groups and filter valid sequences
        groups = df.groupby('sequence_group')
        total_groups += len(groups)  # Add total number of groups for this model
        valid_sequences = groups.filter(lambda x: len(x) >= window_size)
        skipped_groups_for_model = len(groups) - len(valid_sequences.groupby('sequence_group'))
        skipped_groups_total += skipped_groups_for_model

        # Step 6: Create time-windowed data
        def process_group(group):
            group_windows = []
            for start_idx in range(0, len(group) - window_size + 1):
                window = group.iloc[start_idx:start_idx + window_size]

                # Flatten the window
                flattened_window = {}
                for column in window.columns:
                    if column not in ['Datetime', 'time_diff', 'is_continuous', 'sequence_group', 'main_fault']:
                        if pd.api.types.is_numeric_dtype(window[column]):
                            flattened_window.update({f"{column}_{i}": v for i, v in enumerate(window[column].tolist())})
                        else:
                            flattened_window.update({f"{column}_0": window[column].iloc[0]})  # Only take the first non-numeric value

                # Add the latest Datetime in the window
                flattened_window["Datetime"] = window['Datetime'].iloc[-1]

                # Add metadata
                flattened_window["main_fault"] = window['main_fault'].mode()[0] if not window['main_fault'].isna().all() else None
                flattened_window["seriesnumber"] = model_number
                group_windows.append(flattened_window)

            return group_windows

        valid_groups = [group for _, group in valid_sequences.groupby('sequence_group')]
        print(f"Processing {len(valid_groups)} valid groups for model {model_number}...")
        print(f"Skipped groups for model {model_number}: {skipped_groups_for_model}")

        for group in valid_groups:
            time_windows.extend(process_group(group))

    print(f"Total groups across all models: {total_groups}")
    print(f"Total skipped groups across all models: {skipped_groups_total}")

    # Step 4: Process "cleaned_data_806278.parquet" file
    if os.path.exists(cleaned_data_file):
        print(f"Processing cleaned data from: {cleaned_data_file}")
        df_cleaned = pd.read_parquet(cleaned_data_file)
        df_cleaned['Datetime'] = pd.to_datetime(df_cleaned['Datetime'])
        df_cleaned = df_cleaned.sort_values('Datetime').reset_index(drop=True)

        # Identify continuous sequences
        df_cleaned['time_diff'] = df_cleaned['Datetime'].diff().dt.total_seconds()
        df_cleaned['is_continuous'] = (df_cleaned['time_diff'].between(59, 61)) | (df_cleaned.index == 0)
        df_cleaned['sequence_group'] = (~df_cleaned['is_continuous']).cumsum()

        # Determine how many rows to include (10% of the total from time_windows)
        limit = int(len(time_windows) * 0.1)

        # Select rows up to the limit
        cleaned_windows = []
        for _, group in df_cleaned.groupby('sequence_group'):
            if len(group) >= window_size:
                for start_idx in range(0, len(group) - window_size + 1):
                    window = group.iloc[start_idx:start_idx + window_size]

                    # Flatten the window
                    flattened_window = {}
                    for column in window.columns:
                        if column not in ['Datetime', 'time_diff', 'is_continuous', 'sequence_group']:
                            if pd.api.types.is_numeric_dtype(window[column]):
                                flattened_window.update({f"{column}_{i}": v for i, v in enumerate(window[column].tolist())})
                            else:
                                flattened_window.update({f"{column}_0": window[column].iloc[0]})

                    # Add the latest Datetime in the window
                    flattened_window["Datetime"] = window['Datetime'].iloc[-1]
                    flattened_window["main_fault"] = None  # Always None for cleaned data
                    flattened_window["seriesnumber"] = "806278"
                    cleaned_windows.append(flattened_window)

                    # Stop if we've reached the limit
                    if len(cleaned_windows) >= limit:
                        break
            if len(cleaned_windows) >= limit:
                break
        print(f"Added {len(cleaned_windows)} rows from cleaned data.")
        time_windows.extend(cleaned_windows)

    # Combine all flattened windows into a single DataFrame
    all_data = pd.DataFrame(time_windows)

    # Step 7: Normalize numeric columns
    scaler = MinMaxScaler()
    numeric_columns = all_data.select_dtypes(include=['number']).columns
    if not numeric_columns.empty:
        all_data[numeric_columns] = scaler.fit_transform(all_data[numeric_columns])

    # Step 8: Save the prepared data
    all_data.to_parquet(output_file, index=False)

    elapsed_time = time.time() - start_time
    print(f"Total elapsed time: {elapsed_time:.2f} seconds")


In [ ]:
pd.read_parquet("/Users/muhammadhussain/Desktop/Data/filter/around_events_data_806274.parquet")

,Datetime,RTD,1st Suc.,Cond. Air In,Evap. In,Evap. Out,2nd Suc.,Chil. water In,2nd Sump,H.E.,SetPoint,Mains Voltage,State,Type,Event,main_fault
0,2017-03-06 23:10:13,-70.5,-12.8,15.9,-85.1,-74.4,-26.7,17.7,51.9,-40.3,-80,231.2,3,None,None,None
1,2017-03-06 23:11:13,-71.8,-12.6,16.0,-85.0,-75.7,-27.3,17.7,51.7,-40.2,-80,231.7,3,None,None,None
2,2017-03-06 23:12:13,-73.9,-12.6,16.0,-84.9,-77.8,-27.6,17.7,51.5,-39.9,-80,231.1,3,None,None,None
3,2017-03-06 23:13:13,-74.7,-12.6,15.7,-84.9,-81.5,-28.2,17.7,51.5,-39.8,-80,231.2,3,None,None,None
4,2017-03-06 23:14:13,-76.3,-12.6,15.8,-84.9,-84.1,-28.7,17.7,51.4,-39.5,-80,231.4,3,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,2017-03-07 10:25:13,-80.2,-3.3,14.7,-69.7,-82.2,4.2,15.2,32.6,-27.6,-80,232.6,1,None,None,None
676,2017-03-07 10:26:13,-79.7,-34.5,14.8,-69.6,-82.0,3.1,16.9,31.4,-44.7,-80,229.1,3,None,None,None
677,2017-03-07 10:27:13,-79.4,-39.5,15.1,-77.3,-83.2,-32.8,17.2,29.2,-39.7,-80,229.6,3,None,None,None
678,2017-03-07 10:28:13,-79.3,-37.9,15.1,-81.7,-83.5,-46.9,17.1,27.3,-34.2,-80,229.2,3,None,None,None


In [21]:
pd.read_parquet("/Users/muhammadhussain/Desktop/Data/filter/finished.parquet")["main_fault"].unique()

array(['Refrigerant leakage at stage 1', 'High condensation water',
       'electric_wiring', 'compressor_stage_1 malfunctional',
       'instability', None], dtype=object)